In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np

In [14]:
model = keras.models.load_model("clothing-model.h5")

In [12]:
img = load_img('pants.jpg', target_size=(299,299))

x = np.array(img)
X = np.array([x])

X = preprocess_input(X)

In [15]:
preds = model.predict(X)

1/1 [==============================] - 1s 1s/step


In [16]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [17]:
dict(zip(classes, preds[0]))

{'dress': -1.8798643,
 'hat': -4.756312,
 'longsleeve': -2.359532,
 'outwear': -1.089265,
 'pants': 9.903784,
 'shirt': -2.8261797,
 'shoes': -3.6483119,
 'shorts': 3.2411563,
 'skirt': -2.612096,
 't-shirt': -4.852035}

## Convert Keras to TF-Lite

In [20]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('clothing-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpi_of16tg/assets


INFO:tensorflow:Assets written to: /tmp/tmpi_of16tg/assets
2023-11-20 14:38:36.897943: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-20 14:38:36.898050: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-20 14:38:36.898473: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpi_of16tg
2023-11-20 14:38:36.924599: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-20 14:38:36.924660: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpi_of16tg
2023-11-20 14:38:37.010547: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-20 14:38:37.588040: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: /tmp/tmpi_of16tg
2023-11-20 14:38:37.795413: I tensorflow/cc/saved_model/loader.cc:316] SavedModel

In [21]:
import tensorflow.lite as tflite

In [22]:
interpreter = tflite.Interpreter(model_path="clothing-model.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [24]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [31]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -1.8798646,
 'hat': -4.7563095,
 'longsleeve': -2.3595328,
 'outwear': -1.089262,
 'pants': 9.903784,
 'shirt': -2.826181,
 'shoes': -3.6483092,
 'shorts': 3.2411563,
 'skirt': -2.6120954,
 't-shirt': -4.8520336}

## Removing TF dependency

In [32]:
from PIL import Image

In [33]:
with Image.open("pants.jpg") as img:
    img = img.resize((299, 299), Image.NEAREST)

In [46]:
def preprocess_input(x):
    x /= 127.5
    x -= 1.
    return x

In [47]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)

In [48]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [49]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -1.8798646,
 'hat': -4.7563095,
 'longsleeve': -2.3595328,
 'outwear': -1.089262,
 'pants': 9.903784,
 'shirt': -2.826181,
 'shoes': -3.6483092,
 'shorts': 3.2411563,
 'skirt': -2.6120954,
 't-shirt': -4.8520336}

## Simpler way of doing it

In [1]:
#import tensorflow.lite as tflite
import tflite_runtime.interpreter as tflite
from keras_image_helper import create_preprocessor

In [2]:
preprocessor = create_preprocessor("xception", target_size=(299,299))

url = 'http://bit.ly/mlbookcamp-pants'
X = preprocessor.from_url(url)

In [3]:
interpreter = tflite.Interpreter(model_path="clothing-model.tflite")
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

preds = interpreter.get_tensor(output_index)

In [5]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds[0]))

{'dress': -1.8798647,
 'hat': -4.75631,
 'longsleeve': -2.3595335,
 'outwear': -1.0892619,
 'pants': 9.903784,
 'shirt': -2.826181,
 'shoes': -3.6483088,
 'shorts': 3.2411566,
 'skirt': -2.6120954,
 't-shirt': -4.852034}